In [1]:
from util import *
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
print("Data loading...")
city = 'ld'
df_aq = get_aq(city)
df_meo = get_meo(city)
df_station = pd.read_csv("../input/" + city +"_aq_stations.csv")

print("Resampling...")
df_origin = df_resample(df_aq)
df_origin.head()

Data loading...
Resampling...


,utc_time,CO,NO2,O3,PM10,PM2.5,SO2,stationId
0,2017-01-01 00:00:00,NaN,36.6,NaN,44.4,40.0,NaN,CD1
1,2017-01-01 01:00:00,NaN,46.2,NaN,34.4,31.6,NaN,CD1
2,2017-01-01 02:00:00,NaN,38.3,NaN,28.1,24.7,NaN,CD1
3,2017-01-01 03:00:00,NaN,32.8,NaN,24.5,21.2,NaN,CD1
4,2017-01-01 04:00:00,NaN,28.1,NaN,23.0,24.9,NaN,CD1


In [3]:
print("Calculating Top-k nearest stations...")
nears = ['near_1', 'near_2']
near_stations = cal_near_stations(df_station, nears)
df_station_near = pd.concat([df_station, pd.DataFrame(near_stations, columns=nears)], axis=1)

Calculating Top-k nearest stations...
0 BX9
0      0.000000
1      0.000000
14     4.654822
10    10.084194
dtype: float64 

1 BX1
0      0.000000
1      0.000000
14     4.654822
10    10.084194
dtype: float64 

2 BL0
2     0.000000
3     0.536116
5     2.436557
23    3.190417
dtype: float64 

3 CD9
3     0.000000
2     0.536116
23    2.860252
5     2.893064
dtype: float64 

4 CD1
4     0.000000
23    2.801780
16    4.574638
17    4.574638
dtype: float64 

5 CT2
5    0.000000
2    2.436557
3    2.893064
6    2.969668
dtype: float64 

6 CT3
6     0.000000
5     2.969668
18    5.117802
2     5.373555
dtype: float64 

7 CR8
7     0.000000
21    2.195718
5     8.142828
2     8.308433
dtype: float64 

8 GN0
8     0.000000
10    2.357876
9     2.831339
12    2.841780
dtype: float64 

9 GR4
9     0.000000
12    1.670083
8     2.831339
11    3.346244
dtype: float64 

10 GN3
10    0.000000
8     2.357876
12    2.501953
9     3.711309
dtype: float64 

11 GR9
11    0.000000
9     3.346244
8     4

In [5]:
def spatial_step(df, target, nears):
    for near in nears:
        df = pd.merge(df, df[[target, 'stationId','utc_time']], 
                     how='left', suffixes=('', '_'+ near),
                     left_on=['utc_time', near], 
                     right_on=['utc_time', 'stationId'])
    return df

def temporal_step(df, target):
    dfs_ = []
    nulls = []
    for station in df.stationId.unique():
        df_ = df[df.stationId==station]
        df_ = df_.sort_values('utc_time')

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)

        cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
        cols += [target+'_near_1', target+'_near_2', target, 'utc_time']

        df_[target+"_pred"] = df_[cols].mean(axis=1)

        # Update valie
        null_idxs = station_to_null_idxs[station]
        if EVALUATION:
            eval_idxs = station_to_eval_idxs[station]
        df_.loc[null_idxs, target] = df_[target + '_pred']
        if EVALUATION:
            df_.loc[eval_idxs, target] = df_[target + '_pred']

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)
        
        condition = df_[target].isnull() if city=="bj" else df_[target].isnull() & df_['need_prediction']==1
        null = df_[condition].shape[0]
        nulls.append(null)
#         print(N_null, station, target)
        dfs_.append(df_[['utc_time', 'stationId', target]])
    return dfs_, nulls

df_stKNNs = []
EVALUATION = False

target_cols = ['PM2.5', 'PM10', 'O3'] if city=='bj' else ['PM2.5', 'PM10']

for target in target_cols:
    print(target)
    station_to_null_idxs = {}
    station_to_eval_idxs = {}
    MSEs = []

    df_impute = pd.merge(df_origin, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
    for station in df_impute.stationId.unique():
        df_ = df_impute[df_impute.stationId==station]
        df_ = df_.sort_values('utc_time')
        if city=="bj":
            station_to_null_idxs[station] = df_[target].isnull() # for params update
            if EVALUATION:
                eval_condition = (df_.utc_time>='2017-05-01 00:00:00') & (df_.utc_time<='2017-05-01 23:00:00') & (df_.stationId=="tiantan_aq") 
                station_to_eval_idxs[station] = eval_condition # for evaluation
        elif city=="ld":
            station_to_null_idxs[station] = (df_[target].isnull()) & (df_['need_prediction']==1) # for params update

    N_nulls = 999
    while N_nulls>0:
        df_impute = spatial_step(df_impute, target, nears)
        df_imputes, nulls = temporal_step(df_impute, target)
        df_impute = pd.concat(df_imputes)
        df_impute = pd.merge(df_impute, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
        N_nulls = sum(nulls)
        print(N_nulls, nulls)
    
    df_stKNNs.append(df_impute)

PM2.5
1642 [3, 44, 31, 3, 1304, 19, 3, 26, 3, 136, 10, 54, 6, 0, 0, 0, 0, 0, 0]
1273 [0, 0, 0, 0, 1273, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1245 [0, 0, 0, 0, 1245, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1217 [0, 0, 0, 0, 1217, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1189 [0, 0, 0, 0, 1189, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1161 [0, 0, 0, 0, 1161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1133 [0, 0, 0, 0, 1133, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1105 [0, 0, 0, 0, 1105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1077 [0, 0, 0, 0, 1077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1051 [0, 0, 0, 0, 1051, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1027 [0, 0, 0, 0, 1027, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1003 [0, 0, 0, 0, 1003, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
979 [0, 0, 0, 0, 979, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
959 [0, 0, 0, 0, 959, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
941 [0, 0, 0, 0, 941, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
92

78 [0, 0, 0, 0, 78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
74 [0, 0, 0, 0, 74, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
70 [0, 0, 0, 0, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
66 [0, 0, 0, 0, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
62 [0, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
58 [0, 0, 0, 0, 58, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
54 [0, 0, 0, 0, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
50 [0, 0, 0, 0, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
46 [0, 0, 0, 0, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
42 [0, 0, 0, 0, 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
38 [0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
34 [0, 0, 0, 0, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30 [0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
26 [0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
22 [0, 0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18 [0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
14 [0, 0

158 [0, 0, 0, 0, 158, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
154 [0, 0, 0, 0, 154, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
150 [0, 0, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
146 [0, 0, 0, 0, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
142 [0, 0, 0, 0, 142, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
138 [0, 0, 0, 0, 138, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
134 [0, 0, 0, 0, 134, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
130 [0, 0, 0, 0, 130, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
126 [0, 0, 0, 0, 126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
122 [0, 0, 0, 0, 122, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
118 [0, 0, 0, 0, 118, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
114 [0, 0, 0, 0, 114, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
110 [0, 0, 0, 0, 110, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
106 [0, 0, 0, 0, 106, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
102 [0, 0, 0, 0, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
98 [0, 0, 0, 0, 98, 0, 0, 0, 0, 0, 0, 0,

151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 0, 0, 0, 0, 0, 0]
151476 [11652, 11652, 11652, 11652, 11652, 11652, 11652, 11652, 

KeyboardInterrupt: 

## Merge & 检查是否正确

In [18]:
df = df_origin.copy()
# df = pd.merge(df, df_stKNNs[1][['utc_time', 'stationId', 'PM10']], how='left', suffixes=("", "_pred"),
#                   left_on=['utc_time', 'stationId'],
#                   right_on=['utc_time', 'stationId'],
#              )

# if city=='bj':
#     df = pd.merge(df, df_stKNNs[2][['utc_time', 'stationId', 'O3']], how='left', suffixes=("", "_pred"),
#                       left_on=['utc_time', 'stationId'],
#                       right_on=['utc_time', 'stationId'],
#                  )


for i, target in enumerate(target_cols):
    df = pd.merge(df, df_stKNNs[i][['utc_time', 'stationId', target]], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],  right_on=['utc_time', 'stationId'],
                 )
    
    tmp = df[pd.notnull(df[target])]
    MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
    print(target, MAE)
    
    df[target] = df[target + '_pred']
    
df = df[['utc_time'] + target_cols + ['stationId']]
df.to_csv("../input/" + city + "_aq_imputed.csv", index=False)


PM2.5 0.0
PM10 0.0


## Evalution (手动选择某一站点，设置一天空值)

In [ ]:
# condition = (df.stationId=="tiantan_aq") & (df.utc_time>='2017-05-01 00:00:00') & (df.utc_time<='2017-05-01 23:00:00')
# tmp = df[['utc_time','PM10', 'PM10_pred']][condition]
# tmp = tmp.sort_values(by='utc_time')
# # tmp
# tmp.plot(x='utc_time', y=['PM10', 'PM10_pred'], figsize=(16,6), style={'PM10': '-', 'PM10_pred': ':'})

# tmp = df[condition]
# tmp = tmp[pd.notnull(tmp[target])]
# MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
# print(MAE)

In [ ]:
# df = pd.merge(df, df_station, 'left', left_on='stationId', right_on='Station_ID')

# df.longitude = df.longitude.round(1)
# df.latitude = df.latitude.round(1)
# df = pd.merge(df, df_meo, 'left', 
#               left_on=['utc_time', 'longitude', 'latitude'], 
#               right_on=['utc_time', 'longitude', 'latitude']
#              )

# _, bins = pd.cut(df['wind_direction'], bins=16, retbins=True)
# df['wind_direction'] = pd.cut(df['wind_direction'], bins=16, labels=False)

# le_wind_direction = LabelEncoder()
# le_wind_direction.fit(df.station_type)
# df.station_type = le_wind_direction.transform(df.station_type)


# le_stationId = LabelEncoder()
# le_stationId.fit(df.stationId)
# df.stationId = le_stationId.transform(df.stationId)

# # df = pd.get_dummies(df, columns=['stationId'])
# df['year'] = df.utc_time.dt.year
# df['month'] = df.utc_time.dt.month
# df['day'] = df.utc_time.dt.day
# df['hour'] = df.utc_time.dt.hour
# df['weekday'] = df.utc_time.dt.weekday

## Handle with MEO

In [ ]:
df = pd.read_csv("../input/ld_aq_historical.csv", index_col=['Unnamed: 0'])
df.columns = ['utc_time', 'stationId','PM2.5', 'PM10', 'NO2']

In [20]:
df_meo

,humidity,latitude,longitude,pressure,stationName,temperature,utc_time,wind_direction,wind_speed/kph
0,77.90,50.5,-2.0,1024.8100,london_grid_000,9.36,2017-01-01 00:00:00,250.88,23.74
1,78.96,50.6,-2.0,1024.2500,london_grid_001,9.09,2017-01-01 00:00:00,249.86,21.81
2,82.13,50.7,-2.0,1022.5500,london_grid_002,8.30,2017-01-01 00:00:00,245.35,16.08
3,85.30,50.8,-2.0,1020.8600,london_grid_003,7.50,2017-01-01 00:00:00,235.98,10.55
4,88.51,50.9,-2.0,1015.8900,london_grid_004,6.92,2017-01-01 00:00:00,228.21,10.97
5,91.73,51.0,-2.0,1010.9200,london_grid_005,6.33,2017-01-01 00:00:00,221.12,11.57
6,92.89,51.1,-2.0,1008.7900,london_grid_006,6.13,2017-01-01 00:00:00,220.29,13.06
7,93.01,51.2,-2.0,1008.0900,london_grid_007,6.12,2017-01-01 00:00:00,221.89,14.96
8,92.99,51.3,-2.0,1008.1200,london_grid_008,6.19,2017-01-01 00:00:00,223.30,16.64
9,92.64,51.4,-2.0,1009.5800,london_grid_009,6.43,2017-01-01 00:00:00,224.78,17.89
